# Project Title: The Impact of News on the Market
### •	Team Members:
##### 	Rachel Torres, Christian Attard, Jess Alcalde, Nitin Khade
### •	Project Description/Outline:
##### -	We will look at news data and stock data to determine the effects of the news on how the market behaves.
### •	Research Questions to Answer:
##### -	How do news headlines affect the stock market?
##### -	Is there any correlation between certain types of headlines and effects on the market?
##### -	Does negative news affect stocks greater than positive or neutral?
##### -	Can we assign a factor(weighting) to it?

In [22]:
# import dependencies
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

from news_api import api_key
from x_api import x_api_key

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

# pip install vaderSentiment

from news_pull import sentiment_scores
from news_pull import news_pull

In [24]:
wayfairDF = news_pull('Wayfair News')

 There are 30 pages, with 1501 total articles.
         datePublished                                        description  \
0  2019-06-27T10:52:51  The protest triggered a broader backlash again...   
1  2019-06-27T07:52:39  Employees at online home furnishings retailer ...   
2  2019-06-26T23:28:47  Wayfair workers walk out to protest U.S. migra...   
3  2019-06-26T22:04:39  Catch up on the most important news from today...   
4  2019-06-26T21:49:10  BOSTON (AP) - Employees at online home furnish...   

                                           keywords      provider  \
0  wayfair workers walk out,lakeland,protest,ledger     theledger   
1            detention center,wayfair,protest,texas          wthr   
2            wayfair workers walk out,one news page   onenewspage   
3   facebook data value,wayfair walkout,boston,news         wired   
4         wayfair employees,detention center,boston  mysanantonio   

                                               title  \
0  Wayfair workers 

### We'll query the news api to gather news headlines from the web

In [2]:
# parameters for news api
q='politics',
from_param='2019-05-24',
language='en',
sort_by='relevancy',
page = 4,
pageSize = 50

url = f'https://newsapi.org/v2/everything?q={q}&from={from_param}&language={language}&sortBy={sort_by}&pageSize={pageSize}&page={page}&apiKey={api_key}'
response = requests.get(url)

url_2 = f'https://newsapi.org/v2/everything?q=apple&from=2019-05-24&to=2019-06-23&language=en&sortBy=popularity&pageSize={pageSize}&page=3&apiKey={api_key}'
response = requests.get(url_2)
# Generate response into json 

In [3]:
pprint(response.json())

{'code': 'parameterInvalid',
 'message': 'You are trying to request results too far in the past. Your plan '
            'permits you to request articles as far back as 2019-05-28, but '
            'you have requested 2019-05-24. To extend this please upgrade to a '
            'paid plan.',
 'status': 'error'}


### We'll use a different API to gather news data since we have a restriction on the news api

In [4]:
# Query contextual web search API
# Replace the following string value with your valid X-RapidAPI-Key.
Your_X_RapidAPI_Key = x_api_key;

# The query parameters: (update according to your search query)
q = "Wayfair" #the search query
pageNumber = 1 #the number of requested page
pageSize = 50 #the size of a page
autoCorrect = True #autoCorrectspelling
safeSearch = False #filter results for adult content


# test response to query and get count of total items and pages
response_test=requests.get("https://contextualwebsearch-websearch-v1.p.rapidapi.com/api/Search/NewsSearchAPI?q={}&pageNumber={}&pageSize={}&autocorrect={}&safeSearch={}".format(q, pageNumber, pageSize, autoCorrect,safeSearch),
headers={
"X-RapidAPI-Key": Your_X_RapidAPI_Key
}
).json()

#Get the number of items returned
totalCount = response_test["totalCount"];
totalPages = round(totalCount/pageSize)


In [5]:
# generate sample response to look at headers
pprint((response_test['value'][2]))

{'datePublished': '2019-06-28T21:40:05',
 'description': 'If it feels good, do itbut dont think youre changing '
                'anything.',
 'image': {'base64Encoding': None,
           'height': 1767,
           'thumbnail': 'https://contextualwebsearch.com/api/thumbnail/get?value=5606056902504794383',
           'thumbnailHeight': 165,
           'thumbnailWidth': 247,
           'url': 'https://static.politico.com/23/7a/0addf3df4195bbbc8e31f6a5e5a9/190628-shafer-wayfairboycott.jpg',
           'width': 2652},
 'isSafe': True,
 'keywords': 'politico magazine,wayfair boycott,time',
 'language': 'en',
 'provider': {'name': 'politico'},
 'title': 'The <b>Wayfair</b> Boycott Is a Waste of Time',
 'url': 'https://www.politico.com/magazine/story/2019/06/28/wayfair-boycott-political-effectiveness-227251'}


In [6]:
print(f' There are {totalPages} pages, with {totalCount} total articles.')

 There are 30 pages, with 1500 total articles.


In [7]:
# create empty lists to hold variable results
url_list = []
title_list = []
description_list = []
keywords_list = []
provider_list = []
date_list = []

# Load news data into pandas dataframe
# page = 1
for page in range(1,totalPages+1):
    response=requests.get("https://contextualwebsearch-websearch-v1.p.rapidapi.com/api/Search/NewsSearchAPI?q={}&pageNumber={}&pageSize={}&autocorrect={}&safeSearch={}".format(q, page, pageSize, autoCorrect,safeSearch),
headers={"X-RapidAPI-Key": Your_X_RapidAPI_Key}).json()

# print(response)
    try:
        #Go over each resulting item
        for webPage in response["value"]:

#Get the web page metadata
            url = webPage["url"]
            title = webPage["title"]
            description = webPage["description"]
            keywords = webPage["keywords"]
            provider = webPage["provider"]["name"]
            datePublished = webPage["datePublished"]
        
        
            url_list.append(url)
            title_list.append(title)
            description_list.append(description)
            keywords_list.append(keywords)
            provider_list.append(provider)
            date_list.append(datePublished)

#             print(f'printing {title}')    
    except (KeyError,IndexError):
            print('Not found, skipping')


In [8]:
# assign list to dataframe

newsFrame = pd.DataFrame({'datePublished':date_list,'description':description_list,'keywords':keywords_list,'provider':provider_list,'title':title_list,'url':url_list})


In [9]:
newsFrame.head(200)

,datePublished,description,keywords,provider,title,url
0,2019-06-29T05:32:00,Related images to https www <b>wayfair</b> com...,"html https www wayfair com,bed frames",parquo,Https Www <b>Wayfair</b> Com Furniture Sb Bed ...,http://parquo.com/https-www-wayfair-com-furnit...
1,2019-06-29T05:11:56,Online furniture giant <b>Wayfair</b> is the l...,"their firms,employees,s display,s demand,wayfair",startribune,Employees demand their firms display a 'moral ...,http://www.startribune.com/employees-demand-th...
2,2019-06-28T21:40:05,"If it feels good, do itbut dont think youre ch...","politico magazine,wayfair boycott,time",politico,The <b>Wayfair</b> Boycott Is a Waste of Time,https://www.politico.com/magazine/story/2019/0...
3,2019-06-28T17:43:12,Their action showed how workers can wield thei...,"wayfair workers walk out,nation",thenation,<b>Wayfair</b> Workers Walk Out,https://www.thenation.com/article/wayfair-work...
4,2019-06-28T15:12:22,"Employees, especially millennials, feel increa...","millennials,wayfair,workers,google",usatoday,<b>Wayfair</b> walkout: Workers getting comfor...,http://rssfeeds.usatoday.com/~/603782436/0/usa...
5,2019-06-28T11:45:44,"Strange days indeed. Wednesday, more than 100 ...","banner day for ignorance,trump hate trump,wayf...",gopusa,"For some, Trump hate trumps all; <b>Wayfair</b...",http://www.gopusa.com/for-some-trump-hate-trum...
6,2019-06-27T18:09:59,The Jewish Labor Committee (JLC) participated ...,"jewish labor committee,jewish journal,wayfair,...",jewishjournal,Jewish Labor Committee Joined <b>Wayfair</b> W...,http://jewishjournal.com/online/300730/jewish-...
7,2019-06-27T16:51:54,<b>Wayfair</b> employee Madeline Howard says W...,"view transcript wayfair,hide transcript,rough cut",reuters,<b>Wayfair</b> workers walk out to protest U.S...,http://feeds.reuters.com/~r/reuters/INVideoTop...
8,2019-06-27T15:54:43,Employees at online home furnishings retailer ...,"protest furniture,detention center,employees,w...",concordmonitor,<b>Wayfair</b> workers protest furniture sale ...,https://www.concordmonitor.com/Wayfair-workers...
9,2019-06-27T15:48:00,"HHS spokeswoman Evelyn Stauff says it's ""worki...",trump official defends wayfair bed sale to bor...,cnbc,Trump official defends <b>Wayfair</b> bed sale...,https://www.cnbc.com/2019/06/27/trump-official...


In [10]:
newsFrame.dtypes

datePublished    object
description      object
keywords         object
provider         object
title            object
url              object
dtype: object

In [11]:
# clean description and title columns
newsFrame.description = newsFrame.description.str.replace('<b>','')
newsFrame.description = newsFrame.description.str.replace('</b>','')
newsFrame.title = newsFrame.title.str.replace('<b>','')
newsFrame.title = newsFrame.title.str.replace('</b>','')

In [12]:
newsFrame.head()

,datePublished,description,keywords,provider,title,url
0,2019-06-29T05:32:00,Related images to https www wayfair com furnit...,"html https www wayfair com,bed frames",parquo,Https Www Wayfair Com Furniture Sb Bed Frames ...,http://parquo.com/https-www-wayfair-com-furnit...
1,2019-06-29T05:11:56,Online furniture giant Wayfair is the latest t...,"their firms,employees,s display,s demand,wayfair",startribune,Employees demand their firms display a 'moral ...,http://www.startribune.com/employees-demand-th...
2,2019-06-28T21:40:05,"If it feels good, do itbut dont think youre ch...","politico magazine,wayfair boycott,time",politico,The Wayfair Boycott Is a Waste of Time,https://www.politico.com/magazine/story/2019/0...
3,2019-06-28T17:43:12,Their action showed how workers can wield thei...,"wayfair workers walk out,nation",thenation,Wayfair Workers Walk Out,https://www.thenation.com/article/wayfair-work...
4,2019-06-28T15:12:22,"Employees, especially millennials, feel increa...","millennials,wayfair,workers,google",usatoday,Wayfair walkout: Workers getting comfortable p...,http://rssfeeds.usatoday.com/~/603782436/0/usa...


In [13]:
print(totalPages)

30


In [14]:
newsFrame.count()

datePublished    865
description      865
keywords         865
provider         865
title            865
url              865
dtype: int64

In [15]:
# Query sample news description
newsFrame.iloc[7][1]

'Wayfair employee Madeline Howard says Wednesday\'s protest is about making it as "hard as possible" for the U.S. to operate immigration camps. Rough Cut (no reporter narration).}'

In [ ]:
# import stock data 

# Load stock data into notebook as dataframe

In [ ]:
# Merge 2 DataFrames

### About the Scoring (taken from vaderSentiment docs)
The compound score is computed by summing the valence scores of each word in the lexicon, adjusted according to the rules, and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive). This is the most useful metric if you want a single unidimensional measure of sentiment for a given sentence. Calling it a 'normalized, weighted composite score' is accurate.

It is also useful for researchers who would like to set standardized thresholds for classifying sentences as either positive, neutral, or negative. Typical threshold values (used in the literature cited on this page) are:

    positive sentiment: compound score >= 0.05
    neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
    negative sentiment: compound score <= -0.05
The pos, neu, and neg scores are ratios for proportions of text that fall in each category (so these should all add up to be 1... or close to it with float operation). These are the most useful metrics if you want multidimensional measures of sentiment for a given sentence.

In [16]:
# Do sentiment analysis of news data

# Define function to test 
def sentiment_scores_print(sentence):
    score = analyser.polarity_scores(sentence)
    print("{:-<40} \n \n{}".format(sentence, str(score)))
    return score

def sentiment_scores(sentence):
    score = analyser.polarity_scores(sentence)
    return score

# Do test of single string
sentiment_scores_print(newsFrame.iloc[0][1])

Related images to https www wayfair com furniture sb bed frames c html 
 
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [17]:
# add column in newsFrame for sentiment score
newsFrame['sentiment_score'] = ''
newsFrame['sentiment'] = ''

In [18]:
# check dataframe
newsFrame.head()

,datePublished,description,keywords,provider,title,url,sentiment_score,sentiment
0,2019-06-29T05:32:00,Related images to https www wayfair com furnit...,"html https www wayfair com,bed frames",parquo,Https Www Wayfair Com Furniture Sb Bed Frames ...,http://parquo.com/https-www-wayfair-com-furnit...,,
1,2019-06-29T05:11:56,Online furniture giant Wayfair is the latest t...,"their firms,employees,s display,s demand,wayfair",startribune,Employees demand their firms display a 'moral ...,http://www.startribune.com/employees-demand-th...,,
2,2019-06-28T21:40:05,"If it feels good, do itbut dont think youre ch...","politico magazine,wayfair boycott,time",politico,The Wayfair Boycott Is a Waste of Time,https://www.politico.com/magazine/story/2019/0...,,
3,2019-06-28T17:43:12,Their action showed how workers can wield thei...,"wayfair workers walk out,nation",thenation,Wayfair Workers Walk Out,https://www.thenation.com/article/wayfair-work...,,
4,2019-06-28T15:12:22,"Employees, especially millennials, feel increa...","millennials,wayfair,workers,google",usatoday,Wayfair walkout: Workers getting comfortable p...,http://rssfeeds.usatoday.com/~/603782436/0/usa...,,


In [19]:
# add in sentiment analysis to data frame

for i, row in newsFrame.iterrows():
    sentiment_score = float(sentiment_scores(row[1])['compound']) # take sentiment of description of each article
    if(sentiment_score >= 0.05):
        row['sentiment'] = 'positive'
    elif(sentiment_score <= 0.05):
        row['sentiment'] = 'negative'
    else:
        row['sentiment'] = 'neutral'
    row['sentiment_score'] = sentiment_score
#     print((sentiment_score))

In [20]:
# check dataframe to see if sentiment score was added in
newsFrame.head()

,datePublished,description,keywords,provider,title,url,sentiment_score,sentiment
0,2019-06-29T05:32:00,Related images to https www wayfair com furnit...,"html https www wayfair com,bed frames",parquo,Https Www Wayfair Com Furniture Sb Bed Frames ...,http://parquo.com/https-www-wayfair-com-furnit...,0,negative
1,2019-06-29T05:11:56,Online furniture giant Wayfair is the latest t...,"their firms,employees,s display,s demand,wayfair",startribune,Employees demand their firms display a 'moral ...,http://www.startribune.com/employees-demand-th...,0,negative
2,2019-06-28T21:40:05,"If it feels good, do itbut dont think youre ch...","politico magazine,wayfair boycott,time",politico,The Wayfair Boycott Is a Waste of Time,https://www.politico.com/magazine/story/2019/0...,0.4404,positive
3,2019-06-28T17:43:12,Their action showed how workers can wield thei...,"wayfair workers walk out,nation",thenation,Wayfair Workers Walk Out,https://www.thenation.com/article/wayfair-work...,-0.743,negative
4,2019-06-28T15:12:22,"Employees, especially millennials, feel increa...","millennials,wayfair,workers,google",usatoday,Wayfair walkout: Workers getting comfortable p...,http://rssfeeds.usatoday.com/~/603782436/0/usa...,-0.2263,negative


In [ ]:
# Use Matplotlib and stats to generate graphs and look for relationships